---
*NAME - BHAVSAR VISHVA*

*ID - 202318019*

*COURSE - MSC DS*

*SUBJ - BD 7*



---

In [6]:
import numpy as np
import json

In [3]:
!pip install confluent_kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 11.1 MB/s eta 0:00:00


In [4]:
#Consumer
from confluent_kafka import Consumer, KafkaError

In [ ]:
def consume_inventory_data():
    # Kafka consumer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'inventory_group'}

    # Create Kafka consumer
    consumer = Consumer(kafka_config)
    consumer.subscribe(['inventory_orders'])

    # Consume messages
    while True:
        msg = consumer.poll(timeout=1.0)  # Poll for messages
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        inventory_data = json.loads(msg.value().decode('utf-8'))
        print("Received inventory data:", inventory_data)

def consume_delivery_data():
    # Kafka consumer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'delivery_group'}

    # Create Kafka consumer
    consumer = Consumer(kafka_config)
    consumer.subscribe(['delivery_orders'])

    # Consume messages
    while True:
        msg = consumer.poll(timeout=1.0)  # Poll for messages
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        # Process delivery message
        delivery_data = json.loads(msg.value().decode('utf-8'))
        print("Received delivery data:", delivery_data)

In [ ]:
# Consume inventory and delivery data
consume_inventory_data()
consume_delivery_data()

In [ ]:
#message filter
def filter_inventory_message(message):
    """
    Filter inventory messages based on the 'type' field.

    Args:
        message (str): JSON-encoded message.

    Returns:
        bool: True if the message type is 'inventory', otherwise False.
    """
    try:
        data = json.loads(message)
        return data.get('type') == 'inventory'
    except json.JSONDecodeError:
        return False


In [ ]:
def filter_delivery_message(message):
    """
    Filter delivery messages based on the 'type' field.

    Args:
        message (str): JSON-encoded message.

    Returns:
        bool: True if the message type is 'delivery', otherwise False.
    """
    try:
        data = json.loads(message)
        return data.get('type') == 'delivery'
    except json.JSONDecodeError:
        return False

In [ ]:
#Producer
from confluent_kafka import Producer

In [ ]:
def produce_inventory_order():
    # configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    # producer
    producer = Producer(kafka_config)

    # Simulate inventory events data (replace this with actual data source)
    inventory_events = [
        {"type": "inventory", "item_id": "123", "quantity": 10},
        {"type": "inventory", "item_id": "456", "quantity": 20}
    ]

    # Send inventory events to Kafka topic
    for event in inventory_events:
        producer.produce('inventory_orders', json.dumps(event).encode('utf-8'))

    producer.flush()

In [ ]:
def produce_delivery_order():
    # Kafka producer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    # Create Kafka producer
    producer = Producer(kafka_config)

    # Simulate delivery events data (replace this with actual data source)
    delivery_events = [
        {"type": "delivery", "order_id": "1001", "status": "pending"},
        {"type": "delivery", "order_id": "1002", "status": "shipped"}
    ]

    # Send delivery events to Kafka topic
    for event in delivery_events:
        producer.produce('delivery_orders', json.dumps(event).encode('utf-8'))

    producer.flush()

In [ ]:
# Produce inventory and delivery orders
produce_inventory_order()
produce_delivery_order()